In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
from tqdm import tqdm

import torch
import torch.nn as nn 
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

device = 'mps' if torch.backends.mps.is_available() else 'cpu'


#  Loading Data Files

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {}
        self.word2count = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words+=1
        else:
            self.word2count[word]+=1

In [3]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [4]:
def readLangs(lang1, lang2, reverse=False):
    # Read the files and split into line
    lines = open(f'data/{lang1}-{lang2}.txt', encoding='utf-8').\
    read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    
    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

### The full process for preparing the data is:

1. Read text file and split into lines, split lines into pairs
2. Normalize text, filter by length and content
3. Make word lists from sentences in pairs

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)

    print(f'Read {len(pairs)} setennce pairs')

    pairs = filterPairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")

    print('Counting words...')
    for pair in tqdm(pairs, total=len(pairs)):
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words: ")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', reverse=True)
print(random.choice(pairs))

Read 135842 setennce pairs
Trimmed to 11445 sentence pairs
Counting words...


100%|██████████| 11445/11445 [00:00<00:00, 405776.87it/s]

Counted words: 
fra 4601
eng 2991
['elle est privee de sortie', 'she is forbidden to go out']


# Seq2Seq model

## Encoder

In [7]:
class EnocderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1):
        super(EnocderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embed = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        embeddings = self.dropout(self.embed(input))
        output, hidden = self.gru(embeddings)

        return output, hidden

## Decoder

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(DecoderRNN, self).__init__()
        
        self.embed = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        
        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                # Without Teacher forcing: Using its own output as inputs
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()
        
        decoder_outputs = torch.concat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=1)

        return decoder_outputs, decoder_hidden, None # We return None for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embed(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [9]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        